In [ ]:
from redis_manager import StoreRedis
from config import REDIS_STORE_KEY
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA 
from statsmodels.tsa.stattools import adfuller 
from matplotlib import pyplot as plt
from pandas.plotting import autocorrelation_plot
import IPython
import IPython.display
import statsmodels.api as sm


store_redis = StoreRedis()





def ModelArima(Signal,size_train,p ,q = 1, d = 0): #p son los lags de la autocorrelación
    train=Signal[:size_train]
    test=Signal[size_train:]
    resultado_adfuller=adfuller(train)
    Primera_Dif=adfuller(train.diff().dropna())
    Segunda_Dif=adfuller(train.diff().diff().dropna())
        
    modelo = ARIMA(train, order=(p,d,q),)

    modelo_fit = modelo.fit()
    
    tamaño_predicción=len(test)+4
    #output = model_fit.forecast(4)
    predicciones=modelo_fit.get_forecast(tamaño_predicción)
    return train, test, predicciones
    
def plot_series(train, test, predicciones):
    
    conf = predicciones.conf_int(alpha=0.05)
    lower=pd.Series(conf.values[:,0],predicciones.predicted_mean.index)
    high=pd.Series(conf.values[:,1],predicciones.predicted_mean.index)
    
    ax=plt.figure(figsize=(14,6))
    plt.plot(train["Temperature"],label="entrenamiento")
    plt.plot(test["Temperature"],label="testeo")
    # test.plot(legend=True,label="testeo")
    predicciones.predicted_mean.plot(legend=True,label='predicción')
    plt.fill_between(lower.index, lower,high, color='k',alpha=0.15)
    plt.legend(loc="upper left", fontsize=10)
    plt.title('Entrenamiento y Predicción usando ARIMA')


#autocorrelation_plot(temperature_df)
"""
plt.ion()
for i in range(100):
    x = range(i)
    y = range(i)
    # plt.gca().cla() # optionally clear axes
    plt.plot(x, y)
    plt.title(str(i))
    plt.draw()
    plt.pause(1)
    IPython.display.clear_output()
"""

plt.ion()
while True:
    data_train = store_redis.response(REDIS_STORE_KEY, 100)

    temperature_list = [i["temperature"] for i in data_train]
    timestamp_list = [i["timestamp"] for i in data_train]
    temperature_df = pd.DataFrame({"Temperature": temperature_list})   
    
    ax=sm.tsa.acf(temperature_df)
    k=0
    for i in range(0,len(ax)):
        if ax[i] >= 0.5:
            k=k+1
        else:
            None
    train, test, predicciones = ModelArima(temperature_df, 90, p=k)
    # IPython.display.clear_output()
    plot_series(train, test, predicciones )
    plt.pause(1)
    IPython.display.clear_output()
  

  
    


: 